There are two type of data sets, isolated data set and interleaved data set.
The isolated data set is ../data/ptp_isolated_data.csv. The interleaved data set is ../data/ptp_interleaved_data.csv. (PTP stands for Purchase to Pay).
The isolated data set was created by recording the network traffic while only one business process was performed in an isolated manner. In this data all the packets are labeled by the activity and business process case.
The interleaved data set was created by running multiple business processes in parallel. In the resulting data set, the activities' packets are interleaved. In this data only the packets representing the activity start and end are labeled and for those packets we also labeled the case id.

The first step it to observe the data. We focus on the isolated data set.

In [4]:
import os
import case_id_assignment.utilities as util
data_folder = '../data'

In [2]:
isolated_data_path = os.path.join(data_folder, 'ptp_isolated_data.csv')
df = util.load_data_set(isolated_data_path)
df.head()

,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,...,MessageType,MessageAttributes,query_type,session_class,filter_flag,query,tables,event,event_with_roles,noise_event
0,0,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.637118,119,2020-12-22 11:39:44.237118,2020-12-22 11:39:44.237118,192.168.11.2-192.168.11.9 (1),http,...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '...",NaN,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False
1,1,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.688344,153,2020-12-22 11:48:16.548344,2020-12-22 11:48:16.548344,192.168.11.2-192.168.11.9 (1),http,...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E...",NaN,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False
2,2,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.690577,161,2020-12-22 11:48:38.880577,2020-12-22 11:48:38.880577,192.168.11.2-192.168.11.9 (2),http,...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\\r\\n', '...",NaN,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False
3,3,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.985813,224,2020-12-22 12:37:51.535813,2020-12-22 12:37:51.535813,192.168.11.1-192.168.11.2 (7),pgsql,...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",INSERT,pgsql,True,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",['res_users_log'],PgsqlRequest:Simple query:INSERT:['res_users_l...,Odoo Application->db Server/Mail Server: [Pgsq...,False
4,4,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.994598,245,2020-12-22 12:39:19.394598,2020-12-22 12:39:19.394598,192.168.11.2-192.168.11.9 (2),http,...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\\r\\n', '_ws.expert': 'E...",NaN,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False


From observing the first 5 rows we can identify several interesting insights. Each row is representing a packet from the network data. BusinessActivity is the column with the activity label. InstanceNumber is with the case id label. We have three timestamps sniff_time, synthetic_sniff_time and synthetic_sniff_time_str. session_generalized contain the source id, destination ip and session number.
To gain a better description of the data we use the into method.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26006 entries, 0 to 26005
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                26006 non-null  int64 
 1   FileName                  26006 non-null  object
 2   BusinessActivity          26006 non-null  object
 3   InstanceNumber            26006 non-null  int64 
 4   sniff_time                26006 non-null  object
 5   frame.number              26006 non-null  int64 
 6   synthetic_sniff_time      26006 non-null  object
 7   synthetic_sniff_time_str  26006 non-null  object
 8   session_generalized       26006 non-null  object
 9   HighestLayerProtocol      26006 non-null  object
 10  MessageType_WithRole      26006 non-null  object
 11  MessageType               26006 non-null  object
 12  MessageAttributes         26006 non-null  object
 13  query_type                20503 non-null  object
 14  session_class         

Looking at the results from the info method, we see we have 20 columns (potential features) and 26,006 rows (entries).
We can also see most of the data is of type object (str). Very few columns have int or bool data type.
This means we either need to convert the str data to a numeric values, or use method which can analyze string data.

We also identify additional interesting columns (for brevity, we omitted the values since they are very long). MessageAttributes contains the payload of the HTTP message. The column query contains the query executed as part of an PGSQL packet.
The column tables contains the list of tables affected. The column event_with_roles contains a string representation of the source machine, destination machine their roles and the message type (HTTP request, response , PGSQL etc).

The next step is to perform correlation

In [5]:
coor_matrix = df.corr()
coor_matrix['InstanceNumber'].sort_values(ascending=False)

C:\Users\moshe.hadad\AppData\Local\Temp\ipykernel_10752\3375880796.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  coor_matrix = df.corr()


InstanceNumber    1.000000
Unnamed: 0        0.127115
frame.number      0.014340
filter_flag            NaN
noise_event            NaN
Name: InstanceNumber, dtype: float64

Unfortunately, the correlation method works only on numerical values. We either need to convert string data to numerical or use other method for correlation with can take string data.
Looking at the columns again, we can omit FileName, the time stamp columns, the BusinessActivity, frame.number, filter_flag and noise_event. The actual interesting columns we want to check are those :
session_generalized, HighestLayerProtocol, MessageType_WithRole, MessageType, MessageAttributes, query_type, session_class, query, tables, event and event_with_roles.
For the correlation check, we can convert them to categorical value.

In [7]:
new_df = df.copy()
new_df['session_generalized']=df['session_generalized'].astype('category').cat.codes
new_df['HighestLayerProtocol']=df['HighestLayerProtocol'].astype('category').cat.codes
new_df['MessageType_WithRole']=df['MessageType_WithRole'].astype('category').cat.codes
new_df['MessageType']=df['MessageType'].astype('category').cat.codes
new_df['MessageAttributes']=df['MessageAttributes'].astype('category').cat.codes
new_df['query_type']=df['query_type'].astype('category').cat.codes
new_df['session_class']=df['session_class'].astype('category').cat.codes
new_df['query']=df['query'].astype('category').cat.codes
new_df['tables']=df['tables'].astype('category').cat.codes
new_df['event']=df['event'].astype('category').cat.codes
new_df['event_with_roles']=df['event_with_roles'].astype('category').cat.codes

coor_matrix = new_df.corr()
coor_matrix['InstanceNumber'].sort_values(ascending=False)

C:\Users\moshe.hadad\AppData\Local\Temp\ipykernel_10752\2110668116.py:14: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  coor_matrix = new_df.corr()


InstanceNumber          1.000000
Unnamed: 0              0.127115
frame.number            0.014340
MessageAttributes       0.009125
query                   0.006380
tables                  0.003556
event                  -0.000449
event_with_roles       -0.000577
MessageType            -0.000868
HighestLayerProtocol   -0.001234
session_class          -0.001234
MessageType_WithRole   -0.001453
query_type             -0.001914
session_generalized    -0.049040
filter_flag                  NaN
noise_event                  NaN
Name: InstanceNumber, dtype: float64

The second step it to observe the interleaved data set.

In [7]:
interleaved_data_path = os.path.join(data_folder, 'ptp_interleaved_data.csv')
df_in = util.load_data_set(interleaved_data_path)
df_in.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,...,session_class,filter_flag,query,tables,event,event_with_roles,noise_event,real_activity,real_activity_action,real_case_id
0,0,0,27,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.687676,96,2022-08-04 21:07:16.297676,2022-08-04 21:07:16.297676,...,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False,CreatePurchaseRequest Start,Activity Start,399.0
1,1,1,82,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.958633,137,2022-08-04 21:52:26.138633,2022-08-04 21:52:26.138633,...,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False,NoAction,NoAction,NaN
2,2,2,86,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.960536,145,2022-08-04 21:52:45.170536,2022-08-04 21:52:45.170536,...,http,True,NaN,NaN,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,End Point (Employee)->Odoo Application: [HttpR...,False,NoAction,NoAction,NaN
3,3,3,188,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.298610,209,2022-08-04 22:49:06.248610,2022-08-04 22:49:06.248610,...,pgsql,True,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",['res_users_log'],PgsqlRequest:Simple query:INSERT:['res_users_l...,Odoo Application->db Server/Mail Server: [Pgsq...,False,NoAction,NoAction,NaN
4,4,4,258,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.413521,232,2022-08-04 23:08:15.473521,2022-08-04 23:08:15.473521,...,http,True,NaN,NaN,HttpResponse:HTTP/1.0 200 OK\r\n,Odoo Application->End Point (Employee): [HttpR...,False,NoAction,NoAction,NaN


In [8]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0.2              3717 non-null   int64  
 1   Unnamed: 0.1              3717 non-null   int64  
 2   Unnamed: 0                3717 non-null   int64  
 3   FileName                  3717 non-null   object 
 4   BusinessActivity          3717 non-null   object 
 5   InstanceNumber            3717 non-null   int64  
 6   sniff_time                3717 non-null   object 
 7   frame.number              3717 non-null   int64  
 8   synthetic_sniff_time      3717 non-null   object 
 9   synthetic_sniff_time_str  3717 non-null   object 
 10  session_generalized       3717 non-null   object 
 11  HighestLayerProtocol      3717 non-null   object 
 12  MessageType_WithRole      3717 non-null   object 
 13  MessageType               3717 non-null   object 
 14  MessageA

The interleaved data is the data which we will apply the develop method. We load the data to observe it. Fig \ref{fig:interleaved_data} depict the first 5 rows from the interleaved data. and Fig \ref{fig:interleaved_data_info} depics the info of the data. From inspecting the columns we can see that the both data set share most of the columns but there are several differences. In the interleaved data, the BusinessActivity and the InstanceNumber contains garbage data since both the activity and the case id are unknown. We can also notice we have 3 new columns, real\_activity, real\_activity\_action and real\_case\_id. The columns real\_activity\_action holds the labeling of the start and end of an activity i.e. Activity End, Activity Start and NoAction. The real\_activity columns holds the labeling of the activity type e.g CreatePurchaseRequest Start, CreateCallForTender Start, CreatePurchaseRequest End etc. Those columns will be used only for evaluation purposes